In [1]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)

from weighted_accuracy_and_tools import decompose_y, reconstruct_y, weighted_accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd

In [2]:
X_train_path= os.path.join("..", "..", "data","enriched_input", "X_train.csv")
X_train = pd.read_csv(X_train_path, delimiter=',')

X_test_path= os.path.join("..", "..", "data","enriched_input", "X_test.csv")
X_test = pd.read_csv(X_test_path, delimiter=',')
delivery_start = X_test["DELIVERY_START"]

y_train_path= os.path.join("..", "..", "data","enriched_input", "y_train.csv")
y_train = pd.read_csv(y_train_path, delimiter=',')

In [3]:
X_train.set_index("DELIVERY_START", inplace=True)
X_test.set_index("DELIVERY_START", inplace=True)
y_train.set_index("DELIVERY_START", inplace=True)
X_train.index = pd.to_datetime(X_train.index, utc=True)
X_test.index = pd.to_datetime(X_test.index, utc=True)
y_train.index = pd.to_datetime(y_train.index, utc=True)

In [4]:
X_train = X_train.drop(columns=['year'])
X_test = X_test.drop(columns=['year'])

In [5]:
X_train

,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,cold_rate,av_consuption_hour,hour,dayofweek,quarter,month,dayofyear
DELIVERY_START,,,,,,,,,,,,,,,
2022-01-01 01:00:00+00:00,-0.405599,1.143744,0.049156,1.145111,-0.441788,-0.709854,-0.349637,-0.580887,-1.112973,-0.775054,-1.519140,1.002602,-1.045267,-1.287117,-1.406310
2022-01-01 02:00:00+00:00,-0.682678,1.143744,0.049156,1.145111,-0.409506,-0.709854,-0.482002,-0.580887,-1.112973,-1.010691,-1.374581,1.002602,-1.045267,-1.287117,-1.406310
2022-01-01 03:00:00+00:00,-0.810713,1.143744,0.049156,1.145111,-0.366164,-0.709854,-0.614471,-0.580887,-1.112973,-1.565537,-1.230023,1.002602,-1.045267,-1.287117,-1.406310
2022-01-01 04:00:00+00:00,-0.846578,1.143744,0.049156,1.145111,-0.318637,-0.709854,-0.676317,-0.580887,-1.112973,-1.991068,-1.085464,1.002602,-1.045267,-1.287117,-1.406310
2022-01-01 05:00:00+00:00,-0.798790,1.143744,0.049156,1.145111,-0.249289,-0.709854,-0.715292,-0.580887,-1.112973,-2.026920,-0.940906,1.002602,-1.045267,-1.287117,-1.406310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-29 17:00:00+00:00,-0.275482,1.143744,1.010389,0.384129,0.908396,-0.469079,0.924332,-0.399128,-0.492362,-0.085946,0.793798,-0.489343,-1.045267,-0.734099,-0.618926
2023-03-29 18:00:00+00:00,-0.293084,1.143744,1.010389,0.384129,1.143341,-0.669539,0.230275,-0.522000,-0.492362,0.271836,0.938356,-0.489343,-1.045267,-0.734099,-0.618926
2023-03-29 19:00:00+00:00,-0.522752,1.143744,1.010389,0.384129,1.375595,-0.709854,0.011252,-0.580887,-0.492362,0.969350,1.082915,-0.489343,-1.045267,-0.734099,-0.618926


In [6]:
y_train_direction, y_train_magnitude = decompose_y(y_train['spot_id_delta'])

In [18]:
y_train_direction.unique()

array([1, 0])

Classifier

In [7]:
clf = RandomForestClassifier(random_state=42)
sfs_rf = SequentialFeatureSelector(clf, n_features_to_select='auto', direction='forward', scoring='accuracy', cv=5)
sfs_rf.fit(X_train, y_train_direction)
clf.fit(X_train, y_train_direction)
y_pred_direction = clf.predict(X_test)

In [8]:
y_pred_direction

array([0, 0, 0, ..., 0, 0, 0])

Regressor

In [9]:
reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train_magnitude)
y_pred_magnitude = reg.predict(X_test)

In [10]:
y_pred = reconstruct_y(y_pred_direction, y_pred_magnitude)

In [11]:
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

#y_pred.to_csv("../../data/output_y_test/y_result_double_rd_forest2.csv", index = False)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [13]:
y_train_direction, y_train_magnitude = decompose_y(y_train['spot_id_delta'])
y_test_direction, y_test_magnitude = decompose_y(y_test['spot_id_delta'])

In [14]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train_direction)
y_pred_direction = clf.predict(X_test)
reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train_magnitude)
y_pred_magnitude = reg.predict(X_test)

In [15]:
y_pred = reconstruct_y(y_pred_direction, y_pred_magnitude)

In [16]:
rmse_voting = np.sqrt(mean_squared_error(y_test, y_pred))

In [17]:
rmse_voting

19.229675763791843

Classic random forest with original data    

In [1]:
import os
import sys
parent_dir = os.path.dirname(os.getcwd())
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
utils_dir = os.path.join(parent_dir, grandparent_dir, "src", "utils")
sys.path.append(utils_dir)

from weighted_accuracy_and_tools import decompose_y, reconstruct_y, weighted_accuracy_score
from process_data import process_data
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pandas as pd

In [11]:
X_train_path= os.path.join("..", "..", "data","original_input", "X_train_Wwou3IE.csv")
X_train_preprocessed = pd.read_csv(X_train_path, delimiter=',')
X_test_path= os.path.join("..", "..", "data","original_input", "X_test_GgyECq8.csv")
X_test_preprocessed = pd.read_csv(X_test_path, delimiter=',')
y_train_path= os.path.join("..", "..", "data","original_input", "y_train_jJtXgMX.csv")
y_train_preprocessed = pd.read_csv(y_train_path, delimiter=',')
delivery_start = X_test_preprocessed["DELIVERY_START"]
X_train_preprocessed

,DELIVERY_START,load_forecast,coal_power_available,gas_power_available,nucelear_power_available,wind_power_forecasts_average,solar_power_forecasts_average,wind_power_forecasts_std,solar_power_forecasts_std,predicted_spot_price
0,2022-01-01 02:00:00+01:00,49439.0,3386.0,11487.0,44118.0,3035.0,0.0,79.248348,0.000000,NaN
1,2022-01-01 03:00:00+01:00,46511.0,3386.0,11487.0,44118.0,3143.0,0.0,61.776532,0.000000,NaN
2,2022-01-01 04:00:00+01:00,45158.0,3386.0,11487.0,44118.0,3288.0,0.0,44.291112,0.000000,NaN
3,2022-01-01 05:00:00+01:00,44779.0,3386.0,11487.0,44118.0,3447.0,0.0,36.127588,0.000000,NaN
4,2022-01-01 06:00:00+01:00,45284.0,3386.0,11487.0,44118.0,3679.0,0.0,30.983023,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...
10600,2023-03-29 19:00:00+02:00,50814.0,3386.0,11952.0,38320.0,7552.0,651.0,247.408490,7.821622,108.11
10601,2023-03-29 20:00:00+02:00,50628.0,3386.0,11952.0,38320.0,8338.0,109.0,155.795012,2.534054,125.66
10602,2023-03-29 21:00:00+02:00,48201.0,3386.0,11952.0,38320.0,9115.0,0.0,126.884684,0.000000,138.01
10603,2023-03-29 22:00:00+02:00,47967.0,3386.0,11952.0,38320.0,9636.0,0.0,156.669189,0.000000,136.74


In [12]:
X_train = process_data(X_train_preprocessed.copy(deep=True), "predicted_spot_price", None, "standard")
X_test = process_data(X_test_preprocessed.copy(deep=True), "predicted_spot_price", None, "standard")
y_train = process_data(y_train_preprocessed.copy(deep=True), None, None, None)

In [13]:
y_train_direction, y_train_magnitude = decompose_y(y_train['spot_id_delta'])

In [14]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train_direction)
y_pred_direction = clf.predict(X_test)
reg = RandomForestRegressor(random_state=42)
reg.fit(X_train, y_train_magnitude)
y_pred_magnitude = reg.predict(X_test)

In [15]:
y_pred = reconstruct_y(y_pred_direction, y_pred_magnitude)
d = {"DELIVERY_START" : delivery_start, "spot_id_delta" : y_pred.reshape(-1)}
y_pred = pd.DataFrame(data = d)

y_pred.to_csv("../../data/output_y_test/y_result_double_rd_forest3.csv", index = False)

Next to do : 

Sequential features on Rd forest and voting with for classifier : RF, DT and KNN and for regressor : RFR, GBR and KNN

Can you make a voting classifier using Rd forest, DecisionTree and KNN and a vot

Next we take the best model in lagged data